In [1]:
from gladier.client import GladierClient as GladierBaseClient

In [2]:
from flows.phils_beta_raf import flow_definition as flow_phil

class KanzusPhilsClient(GladierBaseClient):
    client_id = 'e6c75d97-532a-4c88-b031-8584a319fa3e'

    gladier_tools = [
        'tools.CreatePhil',
        'tools.DialsStills',
    ]
  #  config_filename = 'c:\\Users\\raves\\Documents\\GitHub\\gladier_kanzus\\bacon1.cfg'    
    flow_definition = flow_phil


phils_client = KanzusPhilsClient()

In [3]:
##hacking over container
from funcx.sdk.client import FuncXClient
fxc = FuncXClient()
from tools.dials_stills import funcx_stills_process as stills_cont
cont_dir =  '/home/rvescovi/.funcx/containers/'
container_name = "dials_v1.simg"
dials_cont_id = fxc.register_container(location=cont_dir+'/'+container_name,container_type='singularity')
stills_cont_fxid = fxc.register_function(stills_cont, container_uuid=dials_cont_id)
##

In [43]:

range_start = 0
range_end = 37400
range_delta = 500

def create_ranges(start,end,delta):
    n_batches = int((range_end - range_start) / range_delta)

    proc_range = []
    for k in range(n_batches):
        k_start = k*range_delta
        k_end = k_start + range_delta - 1
        proc_range.append(f'{str(k_start).zfill(5)}...{str(k_end).zfill(5)}')
    return proc_range

proc_range = create_ranges(range_start,range_end,range_delta)
print(proc_range)

['00000...00499', '00500...00999', '01000...01499', '01500...01999', '02000...02499', '02500...02999', '03000...03499', '03500...03999', '04000...04499', '04500...04999', '05000...05499', '05500...05999', '06000...06499', '06500...06999', '07000...07499', '07500...07999', '08000...08499', '08500...08999', '09000...09499', '09500...09999', '10000...10499', '10500...10999', '11000...11499', '11500...11999', '12000...12499', '12500...12999', '13000...13499', '13500...13999', '14000...14499', '14500...14999', '15000...15499', '15500...15999', '16000...16499', '16500...16999', '17000...17499', '17500...17999', '18000...18499', '18500...18999', '19000...19499', '19500...19999', '20000...20499', '20500...20999', '21000...21499', '21500...21999', '22000...22499', '22500...22999', '23000...23499', '23500...23999', '24000...24499', '24500...24999', '25000...25499', '25500...25999', '26000...26499', '26500...26999', '27000...27499', '27500...27999', '28000...28499', '28500...28999', '29000...2949

In [51]:
conf = {'endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
        'local_endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
        }


data_dir = '/eagle/APSDataAnalysis/SSX/S12/PDL1/B'
proc_dir = data_dir
proc_range_start = 500
proc_range_ends = 1000

proc_range = "{34301..37300}"
flow_input = {
    "input": {
        #Processing variables
        "proc_dir": proc_dir,
        "data_dir": data_dir,

        #Dials specific variables.
        "input_files": f"Bounce_7_{proc_range}.cbf", 
        "input_range": proc_range[1:-1],
        "nproc": 20,
        "beamx": "-214.400",
        "beamy": "218.200",

        # funcX endpoints
        "funcx_ep": conf['endpoint'],
        "funcx_local_ep": conf['local_endpoint'],

        # container hack for stills
        "stills_cont_fxid": stills_cont_fxid
    }
}

In [52]:
flow_input['input']

{'proc_dir': '/eagle/APSDataAnalysis/SSX/S12/PDL1/B',
 'data_dir': '/eagle/APSDataAnalysis/SSX/S12/PDL1/B',
 'input_files': 'Bounce_7_{19001..34300}.cbf',
 'input_range': '19001..34300',
 'nproc': 20,
 'beamx': '-214.400',
 'beamy': '218.200',
 'funcx_ep': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
 'funcx_local_ep': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
 'stills_cont_fxid': 'e14f88c0-600f-40d9-8c26-196f4df75775'}

In [53]:
phils_flow = phils_client.start_flow(flow_input=flow_input)


In [54]:
print(phils_client.get_status(phils_flow['action_id']))

{'action_id': 'b697677d-a9f4-4d64-81ad-33573ef9fc32', 'completion_time': 'None', 'created_by': 'urn:globus:auth:identity:95278182-10a1-11e6-9c7e-7b385f033313', 'details': {'code': 'ActionStarted', 'description': 'State Dials Create Phil of type Action started', 'details': {'input': {'tasks': [{'endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad', 'func': '0839f1ba-972b-4362-a3b0-825b6d5f44ae', 'payload': {'beamx': '-214.400', 'beamy': '218.200', 'data_dir': '/eagle/APSDataAnalysis/SSX/S12/PDL1/B', 'funcx_create_phil_funcx_id': '0839f1ba-972b-4362-a3b0-825b6d5f44ae', 'funcx_ep': '8f2f2eab-90d2-45ba-a771-b96e6d530cad', 'funcx_local_ep': '8f2f2eab-90d2-45ba-a771-b96e6d530cad', 'funcx_stills_process_funcx_id': '798c3395-eea6-4bb2-b2ad-e880f11ee88f', 'input_files': 'Bounce_7_{19001..34300}.cbf', 'input_range': '19001..34300', 'nproc': 20, 'proc_dir': '/eagle/APSDataAnalysis/SSX/S12/PDL1/B', 'stills_cont_fxid': 'e14f88c0-600f-40d9-8c26-196f4df75775'}}]}, 'state_name': 'Dials Create Phil', 'sta

In [3]:
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

class KanzusTriggers:
    def __init__(self, folder_path):
        self.observer = Observer()
        self.folder_path = folder_path

    def run(self):
        print("Kanzus Triggers Started")

        event_handler = Handler()
        self.observer.schedule(event_handler, self.folder_path, recursive = True)
        self.observer.start()
        try:
            while True:
                time.sleep(1)
        except:
            self.observer.stop()
            print("Kanzus Triggers Stopped")
  
        self.observer.join()

class Handler(FileSystemEventHandler):
    @staticmethod
    def on_any_event(event):
        if event.is_directory:
            return None
        elif event.event_type == 'created':

            #event.src_path is the file watchdog found
            # Event is created, you can process it now
            KanzusLogic(event.src_path,f_pattern=None,f_ext=None, n_batch=256)
  

def KanzusLogic(cbf_file,f_pattern=None,f_ext=None, n_batch=256):
        print(cbf_file)

        ##cbf is created
        ##n_batch runs plot
        ##cbf name  %n_batch==0
        ## 

exp = KanzusTriggers('.')
exp.run()

.\setup - Copy.py
.\setup - Copy (2).py
.\setup - Copy (3).py
.\setup - Copy (4).py
.\setup - Copy (5).py
.\setup - Copy (6).py
.\setup - Copy (7).py
.\setup - Copy (8).py
Kanzus Triggers Stopped
